## Adapt section 5, *Learning to balance*, using the Cart Pole environment of Gymnasium

### Main steps to take
- gather samples of the system's dynamics by running a random policy
- learn a local linear model of the system's dynamic around the point $\theta = 0$
- modify the reward function in equation (4) to reflect that in our problem the acceleration input can take only 2 values
- compute a balancing policy by leveraging the local linear model and reward function (e.g. LQR controller)

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd

In [ ]:
env = gym.make("CartPole-v1")
# an observation is a tuple (Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity)
print(env.observation_space)
# there are two possible actions, 0 and 1, which correspond to pushing the cart left or right
print(env.action_space)

In [ ]:
def generate_sample_of_system_dynamics(env, nb_samples):
  """ 
  Returns: a dataframe containing predictors and outcomes
  """
  
  samples = []
  s, _ = env.reset()
  for _ in range(nb_samples):
    a = np.random.randint(0, 2)
    next_s, _, d, _, _ = env.step(a)
    samples.append({
      'x': s[0], 
      'x_dot': s[1], 
      'theta': s[2], 
      'theta_dot': s[3], 
      '2u - 1': 2 * a - 1, # mapping inputs to {-1, 1} rather than {0, 1} is important, because otherwise the linear regression needs a non-zero constant to fit well
      'evolution_x': next_s[0], 
      'evolution_x_dot': next_s[1], 
      'evolution_theta_dot': next_s[3]})
    if d:
      s, _ = env.reset()
    else:
      s = next_s
  return pd.DataFrame(samples)

In [ ]:
env = gym.make("CartPole-v1")
samples = generate_sample_of_system_dynamics(env, 10000)
print(samples.describe())

## Fitting a linear model for the system's dynamics around $\theta=0$

Under the assumption thata the pole's angular velocity is a linear
function of the state and action we use linear regression to find
the coefficients based on sample data

In [ ]:
from statsmodels.api import OLS

small_angle_samples = samples[np.abs(samples["theta"]) < 0.05]
predictors = small_angle_samples[['x', 'x_dot', 'theta', 'theta_dot', '2u - 1']]
outcomes = ['evolution_x_dot', 'evolution_theta_dot', 'evolution_x']
for o in outcomes:
  model = OLS(small_angle_samples[o], predictors)
  fit = model.fit()
  print(f"{o} linear model:")
  print(fit.params)
  print(f"Mean squared error of {o} fit", fit.mse_resid)

## Computing a state-feedback controller according to a discrete-time linear quadratic regulator design

In [ ]:
from control import dlqr

dT = 0.02
A = np.array([
  [1, dT, 0, 0], #x
  [0, 1, -0.014107, 0], #x_dot
  [0, 0, 1, dT], #theta
  [0.000031, 0.000019, 0.315099, 0.999998] #theta_dot
])
B = np.array([
  [0], #x
  [0.195111], #x_dot
  [0], #theta
  [-0.292548] #theta_dot
])
Q = np.diag([125, 50, 1200, 25])
R = np.diag([1.5])

K, S, E = dlqr(A, B, Q, R)
print(K)

## Trying out the learned control policy

In [ ]:
# env = gym.make("CartPole-v1", render_mode='human')
# s, _ = env.reset()
# while True:
#   print(np.dot(-K, s))
#   a = 0 if np.dot(-K, s) < 0.5 else 1
#   s, _, dead, trunc, _ = env.step(a)
#   if dead or trunc:
#     break

In [ ]:
env.close()

## 3. Pick a robot arm description and load it in PyBullet

In [7]:
#We choose the edo arm for its simplicity of use and minimal DOF (6).

arm_name = 'edo_description'

#ur5 could be simple too as S Caron provides Pink implementation example for it


In [5]:
import pybullet
from robot_descriptions.loaders.pybullet import load_robot_description


pybullet build time: Nov 29 2024 16:26:49


In [8]:
pybullet.connect(pybullet.GUI_SERVER)
pybullet.setGravity(0, 0, -9.81)
robot_id = load_robot_description(arm_name, useFixedBase=True)

Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1


UNSUPPORTED (log once): POSSIBLE ISSUE: unit 1 GLD_TEXTURE_INDEX_2D is unloadable and bound to sampler type (Depth) - using zero texture because texture unloadable


In [3]:
initial_joint_positions = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
def reset_to_initial_state():
  for i in range(len(initial_joint_positions)):
    pybullet.resetJointState(
      bodyUniqueId=robot_id,
      jointIndex=i,
      targetValue=initial_joint_positions[i],
      targetVelocity=0
    )
    # pybullet.changeDynamics(robot_id, i, linearDamping=0.0, angularDamping=0.0) # not sure if this is important
    pybullet.setJointMotorControl2(robot_id, i, pybullet.VELOCITY_CONTROL, force=0) # apparently this is really important


## 4. Apply pybullet.TORQUE_CONTROL with PD gains to make the robot go to a desired configuration.

In [5]:
import numpy as np
import time
from tqdm import tqdm

def pd_control(q_desired, v_desired, q_measured, v_measured, Kp, Kd):
    """
    Compute the joint torques using PD control based on desired and measured positions and velocities.

    Parameters:
    q_desired (np.array): Desired joint positions
    v_desired (np.array): Desired joint velocities
    q_measured (np.array): Measured joint positions
    v_measured (np.array): Measured joint velocities
    Kp (np.array): Proportional gains
    Kd (np.array): Derivative gains

    Returns:
    tau (np.array): Joint torques
    """
    # Compute position and velocity errors
    position_error = q_desired - q_measured
    velocity_error = v_desired - v_measured
    
    # Compute torques using PD control
    tau = Kp * position_error + Kd * velocity_error
    return tau

def control_to_desired_configuration(q_desired, v_desired, eps=1e-4):
  # Get the number of joints in the robot
  nq = pybullet.getNumJoints(robot_id)
  # Initialize the PD control gains (these can be tuned)
  Kp = 20 * np.ones((nq,))  # Proportional gains
  Kd = 0.05 * Kp # Derivative gains
  for step in range(1000):
      # Get the current joint states (positions and velocities)
      joint_states = pybullet.getJointStates(robot_id, list(range(nq)))
      q_measured = np.array([s[0] for s in joint_states])
      v_measured = np.array([s[1] for s in joint_states])

      if np.max(np.abs(q_measured - q_desired)) < eps and np.max(np.abs(v_measured - v_desired)) < eps:
        return

      # Compute the joint torques using the PD controller
      tau = np.clip(pd_control(q_desired, v_desired, q_measured, v_measured, Kp, Kd), -50, 50)

      # Apply the torques to the robot joints
      for i in range(nq):
        pybullet.setJointMotorControl2(
          bodyIndex=robot_id, 
          jointIndex=i, 
          controlMode=pybullet.TORQUE_CONTROL, 
          force=tau[i])

      # Step the simulation forward
      pybullet.stepSimulation()

  0%|          | 3/1000 [00:00<01:51,  8.98it/s]

[0.5 0.5 0.5 0.5 0.5 0.5]


 10%|█         | 103/1000 [00:08<01:14, 11.97it/s]

[0.02434907 0.03278645 0.06304277 0.04146687 0.24941721 0.5       ]


 20%|██        | 203/1000 [00:16<01:05, 12.16it/s]

...

 80%|████████  | 803/1000 [01:13<00:17, 11.32it/s]

[0.02578147 0.02782931 0.06092004 0.0417043  0.25085994 0.5       ]


 90%|█████████ | 903/1000 [01:22<00:08, 11.74it/s]

[0.02578147 0.02782931 0.06092004 0.0417043  0.25085994 0.5       ]


100%|██████████| 1000/1000 [01:30<00:00, 11.03it/s]


In [ ]:
reset_to_initial_state()
control_to_desired_configuration(np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), np.array([0, 0, 0, 0, 0, 0]))

## 5. Moving the end-effector of the robot arm along a horizontal line

The line will be defined by two points $ p_1 = (0.1, 0.3, 0) $ and $ p_2 = (0, 0.3, 0.1) $

In [ ]:
def move_end_effector_along_trajectory(traj, eff_idx):
  for target in traj:
    q_desired = pybullet.calculateInverseKinematics(
      bodyIndex=robot_id,
      endEffectorLinkIndex=eff_idx, 
      targetPosition=target
    )
    control_to_desired_configuration(q_desired, np.zeros_like(q_desired))


In [ ]:
reset_to_initial_state()
time.sleep(1)
p1 = np.array((0.5, 0, 0.8))
p2 = np.array((0, 0.5, 0.8))
traj = p1 + np.outer(np.linspace(0, 1, 100), p2 - p1)
move_end_effector_along_trajectory(traj, 5)

## 6. Modify the URDF file of the robot description to add a pendulum at the end effector of the arm

In [1]:
import os

from robot_descriptions import edo_description
urdf_file_path = edo_description.URDF_PATH

def addpendulum_urdf(urdf_file_path):
    # Read the URDF file
    with open(urdf_file_path, 'r') as file:
        urdf_data = file.readlines()

    # Remove trailing blank lines and the </robot> tag
    while urdf_data and urdf_data[-1].strip() == "":  # Remove blank lines
        urdf_data.pop()
    # Remove the last line (`</robot>`)
    if urdf_data[-1].strip() == "</robot>":
        urdf_data.pop()


    # The pendulum components to add
    pendulum_urdf = """
  <!-- Pendulum Link (Thin cylinder attached to link_6) -->
  <link name="pendulum_link">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="0.0083" ixy="0" ixz="0" iyy="0.0000" iyz="0" izz="0.0083" /> <!-- ixx =izz = m * l **2 /3, rest is 0 (source wiki)  -->
    </inertial>
    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
       <cylinder length="0.5" radius="0.01"/> <!-- Thin cylinder, length 0.5m, radius 0.01m -->
      </geometry>
      <material name="">
        <color rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
       <cylinder length="0.5" radius="0.01"/> <!-- Thin cylinder, length 0.5m, radius 0.01m -->
      </geometry>
    </collision>
  </link>

  <!-- Pendulum Mass at the end of the pendulum_link -->
  <link name="pendulum_mass">
    <inertial>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <mass value="0.5" /> <!-- Mass m at the end -->
      <inertia ixx="0.001" ixy="0" ixz="0" iyy="0.0005" iyz="0" izz="0.0005" /> <!-- ixx = iyy=izz = 2 *m * r **2 /5 , rest is 0 (source wiki)   -->
    </inertial>
    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius ="0.05" /> <!-- Small sphere representing the mass, radius r -->
      </geometry>
      <material name="">
        <color rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius="0.05" />
      </geometry>
    </collision>
  </link>

  <!-- Pendulum Revolute Joint (connects pendulum_link to link_6) -->
  <joint name="pendulum_joint" type="fixed">
    <parent link="link_6" /> <!-- Attach to link_6 -->
    <child link="pendulum_link" />
    <origin xyz="0 0 -0.25" rpy="0 0 0" /> <!-- Positioning pendulum at link_6's end -->
    <axis xyz="0 1 0" /> <!-- Swinging along Y-axis -->
    <limit lower="0" upper="3.1416" effort="0" velocity="0" /> <!-- Passive joint with zero effort -->
    <dynamics damping="0.1" friction="0.1" />
  </joint>

  <!-- Pendulum Mass Joint (fixes mass to the end of the pendulum) -->
  <joint name="pendulum_mass_joint" type="fixed">
    <parent link="pendulum_link" />
    <child link="pendulum_mass" />
    <origin xyz="0 0 -0.5" rpy="0 0 0" /> <!-- Attach mass at the tip of pendulum -->
  </joint>
"""
    # Append the pendulum components to the URDF data
    urdf_data.append(pendulum_urdf)

    # Add the </robot> tag back
    urdf_data.append("</robot>\n")

    # Write the modified URDF back to the file
    with open(urdf_file_path, 'w') as file:
        file.writelines(urdf_data)

    print("URDF file has been updated with pendulum components.")






In [2]:
addpendulum_urdf(urdf_file_path)

pop !
URDF file has been updated with pendulum components.


Watch out, you might want to get the initial file back, the next function helps to do so. You might also do it by hand by clicking on the following link and removing the links and joints corresponding to the pendulum

In [14]:
print(urdf_file_path) #run to get yours

/Users/theo/.cache/robot_descriptions/edo_sim/robots/edo_sim.urdf


In [13]:
import os

def undo_modify_urdf(urdf_file_path):
    # Read the URDF file
    with open(urdf_file_path, 'r') as file:
        urdf_data = file.readlines()

    # Remove trailing blank lines and the </robot> tag
    while urdf_data and urdf_data[-1].strip() == "":  # Remove blank lines
        urdf_data.pop()
    
    # Remove the last line </robot> if it exists
    if urdf_data and urdf_data[-1].strip() == "</robot>":
        urdf_data.pop()

    # Identify and remove the pendulum components and their comments
    pendulum_tags = [
        "<link name=\"pendulum_link\">",
        "<link name=\"pendulum_mass\">",
        "<joint name=\"pendulum_joint\" type=\"fixed\">",
        "<joint name=\"pendulum_mass_joint\" type=\"fixed\">",
        "<!-- Pendulum Link (Thin cylinder attached to link_6) -->",
        "<!-- Pendulum Mass at the end of the pendulum_link -->",
        "<!-- Pendulum Revolute Joint (connects pendulum_link to link_6) -->"
    ]
    
    pendulum_lines_to_remove = []

    # Find the lines related to pendulum components and mark them for removal
    in_pendulum_block = False
    for i, line in enumerate(urdf_data):
        # We want to capture lines that are part of the pendulum section or its comments
        if any(tag in line for tag in pendulum_tags):
            in_pendulum_block = True
        if in_pendulum_block:
            pendulum_lines_to_remove.append(i)
        # End of a pendulum link, mark end of block
        if "</link>" in line and in_pendulum_block:
            in_pendulum_block = False

    # Remove the lines related to the pendulum (including comments)
    for index in reversed(pendulum_lines_to_remove):
        urdf_data.pop(index)

    # Add the closing </robot> tag back to the file if it's missing
    urdf_data.append("</robot>\n")

    # Write the modified URDF back to the file
    with open(urdf_file_path, 'w') as file:
        file.writelines(urdf_data)

    print("URDF file has been reverted to its original state (pendulum removed, comments removed).")





URDF file has been reverted to its original state (pendulum removed, comments removed).


In [ ]:
#run to get back the initial file after running addpendulum_urdf
#undo_modify_urdf(urdf_file_path) 

In [1]:
#you can now load the updated robot 
# (I don't know how to do otherwise than restarting the kernel if I had previously launched a physics server for the previous part)
arm_name = 'edo_description'

import pybullet
from robot_descriptions.loaders.pybullet import load_robot_description
pybullet.connect(pybullet.GUI_SERVER)
pybullet.setGravity(0, 0, -9.81)
robot_id = load_robot_description(arm_name, useFixedBase=True)

pybullet build time: Nov 29 2024 16:26:49


Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
b3Printf: PosixSharedMemory::releaseSharedMemory removed shared memory
b3Printf: PosixSharedMemory::releaseSharedMemory detached shared memory

b3Printf: PosixSharedMemory::releaseSharedMemory removed shared memory
b3Printf: PosixSharedMemory::releaseSharedMemory detached shared memory

MotionThreadFunc thread started


In [ ]:
#and even play with the now different dynamics of the controller

In [3]:
import numpy as np
import time
from tqdm import tqdm
initial_joint_positions = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0,0.0,0.0]
def reset_to_initial_state():
  for i in range(len(initial_joint_positions)):
    pybullet.resetJointState(
      bodyUniqueId=robot_id,
      jointIndex=i,
      targetValue=initial_joint_positions[i],
      targetVelocity=0
    )
    # pybullet.changeDynamics(robot_id, i, linearDamping=0.0, angularDamping=0.0) # not sure if this is important
    pybullet.setJointMotorControl2(robot_id, i, pybullet.VELOCITY_CONTROL, force=0) # apparently this is really important


def pd_control(q_desired, v_desired, q_measured, v_measured, Kp, Kd):
    """
    Compute the joint torques using PD control based on desired and measured positions and velocities.

    Parameters:
    q_desired (np.array): Desired joint positions
    v_desired (np.array): Desired joint velocities
    q_measured (np.array): Measured joint positions
    v_measured (np.array): Measured joint velocities
    Kp (np.array): Proportional gains
    Kd (np.array): Derivative gains

    Returns:
    tau (np.array): Joint torques
    """
    # Compute position and velocity errors
    position_error = q_desired - q_measured
    velocity_error = v_desired - v_measured
    
    # Compute torques using PD control
    tau = Kp * position_error + Kd * velocity_error
    return tau

reset_to_initial_state()

# Get the number of joints in the robot
nq = pybullet.getNumJoints(robot_id)

# Set desired joint positions and velocities (targets for the PD control)
q_desired = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5,0.5,0])  # Desired joint positions (radians), added for the last 2 joints
v_desired = np.zeros_like(q_desired)  # Desired joint velocities (radians/sec)

time.sleep(1)

# Initialize the PD control gains (these can be tuned
Kp = 20 * np.ones((nq,))  # Proportional gains
Kd = 0.05 * Kp # Derivative gains
for step in tqdm(range(1000)):
    # Get the current joint states (positions and velocities)
    joint_states = pybullet.getJointStates(robot_id, list(range(nq)))
    q_measured = np.array([s[0] for s in joint_states])
    v_measured = np.array([s[1] for s in joint_states])

    # Compute the joint torques using the PD controller
    tau = np.clip(pd_control(q_desired, v_desired, q_measured, v_measured, Kp, Kd), -50, 50)

    if step % 100 == 0:
      print(np.abs(q_measured - q_desired))

    # Apply the torques to the robot joints
    for i in range(nq):
      pybullet.setJointMotorControl2(
        bodyIndex=robot_id, 
        jointIndex=i, 
        controlMode=pybullet.TORQUE_CONTROL, 
        force=tau[i])

    # Step the simulation forward
    pybullet.stepSimulation()

    # Optional: Slow down the simulation for visualization or debugging
    time.sleep(1.0 / 240)

  0%|          | 3/1000 [00:00<01:42,  9.72it/s]

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0. ]


 10%|█         | 102/1000 [00:09<01:12, 12.47it/s]

[ 2.0394658  13.67434275  0.76152051  0.16703117  0.66380499  0.42334593
  0.5         0.        ]


 20%|██        | 203/1000 [00:18<01:12, 10.97it/s]

[13.11098993 24.00096057  0.51529001  0.26477651  0.85502472  0.38137854
  0.5         0.        ]


 20%|██        | 205/1000 [00:19<01:14, 10.70it/s]


KeyboardInterrupt: 